In [1]:
import sys
import os
notebook_path = os.getcwd()
parent_path = os.path.abspath(os.path.join(notebook_path, ".."))
sys.path.append(parent_path)

import matplotlib.pyplot as plt

from modules.dataset_func import get_X_from_sim_grid
from modules.dataset_func import get_freqs_from_sim_grid

import numpy as np

import torch
import torch.nn as nn
import torch.nn.functional as F

In [29]:
#import data

sim_grid_path = r"C:\Users\Antoine\Documents\master\PA\abacus\sim_plaque\run_sims\outputs\20250326191527"

X = get_X_from_sim_grid(sim_grid_path, ["distances", "rayons", "plaque_epaisseurs"])
y = get_freqs_from_sim_grid(sim_grid_path, mode="1")


c:\Users\Antoine\Documents\master\PA\abacus\sim_plaque\ANNs\tools.py:8: ParserWarning: Length of header or names does not match length of data. This leads to a loss of data with index_col=False.
  df = pd.read_csv(csv_path,index_col=False, dtype=str)
c:\Users\Antoine\Documents\master\PA\abacus\sim_plaque\ANNs\tools.py:8: ParserWarning: Length of header or names does not match length of data. This leads to a loss of data with index_col=False.
  df = pd.read_csv(csv_path,index_col=False, dtype=str)


In [30]:
#normalisation
from sklearn.preprocessing import StandardScaler

scaler_X = StandardScaler()
X_scaled = scaler_X.fit_transform(X)

scaler_y = StandardScaler()
y_scaled = scaler_y.fit_transform(y.reshape(-1, 1)).squeeze()

print(X)
print(X_scaled)
print("\n")
print(y)
print(y_scaled)

[[25.   4.5  1. ]
 [25.   3.   1. ]]
[[ 0.  1.  0.]
 [ 0. -1.  0.]]


[26.618 26.016]
[ 1. -1.]


In [34]:
X_scaled = torch.from_numpy(X_scaled.astype(np.float32))
y_scaled = torch.from_numpy(y_scaled.astype(np.float32))

print(X_scaled.shape)
print(y_scaled.shape)

torch.Size([2, 3])
torch.Size([2])


In [35]:
class MLP_freq(nn.Module):
    def __init__(self):
        super(MLP_freq, self).__init__()
        self.fc1 = nn.Linear(3, 8)
        self.fc2 = nn.Linear(8, 4)
        self.fc3 = nn.Linear(4, 1)  

    def forward(self, x):
        x = F.tanh(self.fc1(x))
        x = F.tanh(self.fc2(x))
        x = self.fc3(x)

        return x

In [36]:
model_frequ = MLP_freq()
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model_frequ.parameters(), lr=0.001)


num_epochs = 100

# Boucle d'entraînement
for epoch in range(num_epochs):
    model_frequ.train()  # mode entraînement 

    # Forward
    pred = model_frequ(X_scaled)
    loss = criterion(pred, y_scaled)

    # Backward
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    # Affichage de la perte
    print(f"Epoch {epoch+1}/{num_epochs} - Loss: {loss.item():.4f}")

Epoch 1/100 - Loss: 1.4748
Epoch 2/100 - Loss: 1.4686
Epoch 3/100 - Loss: 1.4625
Epoch 4/100 - Loss: 1.4564
Epoch 5/100 - Loss: 1.4504
Epoch 6/100 - Loss: 1.4444
Epoch 7/100 - Loss: 1.4385
Epoch 8/100 - Loss: 1.4327
Epoch 9/100 - Loss: 1.4270
Epoch 10/100 - Loss: 1.4213
Epoch 11/100 - Loss: 1.4157
Epoch 12/100 - Loss: 1.4102
Epoch 13/100 - Loss: 1.4047
Epoch 14/100 - Loss: 1.3993
Epoch 15/100 - Loss: 1.3940
Epoch 16/100 - Loss: 1.3887
Epoch 17/100 - Loss: 1.3835
Epoch 18/100 - Loss: 1.3784
Epoch 19/100 - Loss: 1.3733
Epoch 20/100 - Loss: 1.3683
Epoch 21/100 - Loss: 1.3634
Epoch 22/100 - Loss: 1.3585
Epoch 23/100 - Loss: 1.3538
Epoch 24/100 - Loss: 1.3490
Epoch 25/100 - Loss: 1.3444
Epoch 26/100 - Loss: 1.3398
Epoch 27/100 - Loss: 1.3352
Epoch 28/100 - Loss: 1.3308
Epoch 29/100 - Loss: 1.3263
Epoch 30/100 - Loss: 1.3220
Epoch 31/100 - Loss: 1.3177
Epoch 32/100 - Loss: 1.3135
Epoch 33/100 - Loss: 1.3093
Epoch 34/100 - Loss: 1.3051
Epoch 35/100 - Loss: 1.3011
Epoch 36/100 - Loss: 1.2970
E

c:\Users\Antoine\anaconda3\envs\Baseline\Lib\site-packages\torch\nn\modules\loss.py:610: UserWarning: Using a target size (torch.Size([2])) that is different to the input size (torch.Size([2, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


X = torch.from_numpy(X.astype(np.float32))
y = torch.from_numpy(y.astype(np.float32))